# Continuous training with TFX and Google Cloud AI Platform

## Learning Objectives

1.  Use the TFX CLI to build a TFX pipeline.
2.  Deploy a TFX pipeline version without tuning to a hosted AI Platform Pipelines instance.
3.  Create and monitor a TFX pipeline run using the TFX CLI.
4.  Deploy a new TFX pipeline version with tuning enabled to a hosted AI Platform Pipelines instance.
5.  Create and monitor another TFX pipeline run directly in the KFP UI.

In this lab, you use utilize the following tools and services to deploy and run a TFX pipeline on Google Cloud that automates the development and deployment of a TensorFlow 2.3 WideDeep Classifer to predict forest cover from cartographic data:

* The [**TFX CLI**](https://www.tensorflow.org/tfx/guide/cli) utility to build and deploy a TFX pipeline.
* A hosted [**AI Platform Pipeline instance (Kubeflow Pipelines)**](https://www.tensorflow.org/tfx/guide/kubeflow) for TFX pipeline orchestration.
* [**Dataflow**](https://cloud.google.com/dataflow) jobs for scalable, distributed data processing for TFX components.
* A [**AI Platform Training**](https://cloud.google.com/ai-platform/) job for model training and flock management for parallel tuning trials. 
* [**AI Platform Prediction**](https://cloud.google.com/ai-platform/) as a model server destination for blessed pipeline model versions.
* [**CloudTuner**](https://www.tensorflow.org/tfx/guide/tuner#tuning_on_google_cloud_platform_gcp) and [**AI Platform Vizier**](https://cloud.google.com/ai-platform/optimizer/docs/overview) for advanced model hyperparameter tuning using the Vizier algorithm.

You will then create and monitor pipeline runs using the TFX CLI as well as the KFP UI.

### Setup

#### Update lab environment PATH to include TFX CLI and skaffold

In [ ]:
# import yaml

# Set `PATH` to include the directory containing TFX CLI and skaffold.
# PATH = %env PATH
#%env PATH=/home/jupyter/.local/bin:{PATH}

In [32]:
from google.cloud import aiplatform

#### Validate lab package version installation

In [ ]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"
!python -c "import kfp; print('KFP version: {}'.format(kfp.__version__))"

**Note**: this lab was built and tested with the following package versions:

`TFX version: 1.4.0`  
`KFP version: 1.8.1`

(Optional) If running the above command results in different package versions or you receive an import error, upgrade to the correct versions by running the cell below:

In [ ]:
#%pip install --upgrade --user tfx=1.4.0
#%pip install --upgrade --user kfp==1.8.1

Note: you may need to restart the kernel to pick up the correct package versions.

#### Validate creation of AI Platform Pipelines cluster

Navigate to [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

Note you may have already deployed an AI Pipelines instance during the Setup for the lab series. If so, you can proceed using that instance. If not:

**1.  Create or select an existing Kubernetes cluster (GKE) and deploy AI Platform**. Make sure to select `"Allow access to the following Cloud APIs https://www.googleapis.com/auth/cloud-platform"` to allow for programmatic access to your pipeline by the Kubeflow SDK for the rest of the lab. Also, provide an `App instance name` such as "tfx" or "mlops". 

Validate the deployment of your AI Platform Pipelines instance in the console before proceeding.

## Review: example TFX pipeline design pattern for Google Cloud
The pipeline source code can be found in the `pipeline` folder.

In [2]:
%cd pipeline_vertex

/home/jupyter/asl-ml-immersion/notebooks/tfx_pipelines/pipeline/solutions/pipeline_vertex


In [3]:
!ls -la

total 96
drwxr-xr-x 5 jupyter jupyter  4096 Dec  6 22:35 .
drwxr-xr-x 5 jupyter jupyter  4096 Dec  6 22:34 ..
drwxr-xr-x 2 jupyter jupyter  4096 Dec  6 20:28 .ipynb_checkpoints
-rw-r--r-- 1 jupyter jupyter   118 Dec  6 19:19 Dockerfile
-rw-r--r-- 1 jupyter jupyter 14774 Dec  6 22:28 Untitled.ipynb
drwxr-xr-x 2 jupyter jupyter  4096 Dec  6 22:29 __pycache__
-rw-r--r-- 1 jupyter jupyter  1092 Dec  6 22:34 config.py
-rw-r--r-- 1 jupyter jupyter  1090 Dec  3 19:14 features.py
-rw-r--r-- 1 jupyter jupyter 10380 Dec  4 01:18 model.py
-rw-r--r-- 1 jupyter jupyter  5318 Dec  6 21:38 pipeline.py
-rw-r--r-- 1 jupyter jupyter  2182 Dec  3 19:14 preprocessing.py
-rw-r--r-- 1 jupyter jupyter  1325 Dec  6 22:35 runner.py
drwxr-xr-x 2 jupyter jupyter  4096 Dec  3 19:14 schema
-rw-r--r-- 1 jupyter jupyter 19807 Dec  6 21:42 tfxcovertype.json


The `config.py` module configures the default values for the environment specific settings and the default values for the pipeline runtime parameters. 
The default values can be overwritten at compile time by providing the updated values in a set of environment variables. You will set custom environment variables later on this lab.

The `pipeline.py` module contains the TFX DSL defining the workflow implemented by the pipeline.

The `preprocessing.py` module implements the data preprocessing logic  the `Transform` component.

The `model.py` module implements the training, tuning, and model building logic for the `Trainer` and `Tuner` components.

The `runner.py` module configures and executes `KubeflowDagRunner`. At compile time, the `KubeflowDagRunner.run()` method converts the TFX DSL into the pipeline package in the [argo](https://argoproj.github.io/argo/) format for execution on your hosted AI Platform Pipelines instance.

The `features.py` module contains feature definitions common across `preprocessing.py` and `model.py`.


## Exercise: build your pipeline with the TFX CLI

You will use TFX CLI to compile and deploy the pipeline. As explained in the previous section, the environment specific settings can be provided through a set of environment variables and embedded into the pipeline package at compile time.

### Configure your environment resource settings

Update  the below constants  with the settings reflecting your lab environment. 

- `GCP_REGION` - the compute region for AI Platform Training, Vizier, and Prediction.
- `ARTIFACT_STORE` - An existing GCS bucket. You can use any bucket or use the GCS bucket created during installation of AI Platform Pipelines. The default bucket name will contain the `kubeflowpipelines-` prefix.

In [ ]:
# Use the following command to identify the GCS bucket for metadata and pipeline storage.
!gsutil ls

In [ ]:
!gsutil ls gs://qwiklabs-gcp-04-05130841e161-vertex/

- `ENDPOINT` - set the `ENDPOINT` constant to the endpoint to your AI Platform Pipelines instance. The endpoint to the AI Platform Pipelines instance can be found on the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console. Open the *SETTINGS* for your instance and use the value of the `host` variable in the *Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD* section of the *SETTINGS* window. The format is `'...pipelines.googleusercontent.com'`.

In [97]:
# TODO: Set your environment resource settings here for GCP_REGION, ARTIFACT_STORE_URI, ENDPOINT, and CUSTOM_SERVICE_ACCOUNT.
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
ARTIFACT_STORE = f"gs://{PROJECT_ID}"

In [98]:
# Set your resource settings as environment variables. These override the default values in pipeline/config.py.
%env REGION={REGION}
%env ARTIFACT_STORE={ARTIFACT_STORE}
%env PROJECT_ID={PROJECT_ID}

env: REGION=us-central1
env: ARTIFACT_STORE=gs://qwiklabs-gcp-04-05130841e161
env: PROJECT_ID=qwiklabs-gcp-04-05130841e161


In [47]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ # || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-gcp-04-05130841e161/


### Set the compile time settings to first create a pipeline version without hyperparameter tuning

Default pipeline runtime environment values are configured in the pipeline folder `config.py`. You will set their values directly below:

* `PIPELINE_NAME` - the pipeline's globally unique name. For each pipeline update, each pipeline version uploaded to KFP will be reflected on the `Pipelines` tab in the `Pipeline name > Version name` dropdown in the format `PIPELINE_NAME_datetime.now()`.

* `MODEL_NAME` - the pipeline's unique model output name for AI Platform Prediction. For multiple pipeline runs, each pushed blessed model will create a new version with the format `'v{}'.format(int(time.time()))`.

* `DATA_ROOT_URI` - the URI for the raw lab dataset `gs://workshop-datasets/covertype/small`.

* `CUSTOM_TFX_IMAGE` - the image name of your pipeline container build by skaffold and published by `Cloud Build` to `Cloud Container Registry` in the format `'gcr.io/{}/{}'.format(PROJECT_ID, PIPELINE_NAME)`.

* `RUNTIME_VERSION` - the TensorFlow runtime version. This lab was built and tested using TensorFlow `2.3`.

* `PYTHON_VERSION` - the Python runtime version. This lab was built and tested using Python `3.7`.

* `USE_KFP_SA` - The pipeline can run using a security context of the GKE default node pool's service account or the service account defined in the `user-gcp-sa` secret of the Kubernetes namespace hosting Kubeflow Pipelines. If you want to use the `user-gcp-sa` service account you change the value of `USE_KFP_SA` to `True`. Note that the default AI Platform Pipelines configuration does not define the `user-gcp-sa` secret.

* `ENABLE_TUNING` - boolean value indicating whether to add the `Tuner` component to the pipeline or use hyperparameter defaults. See the `model.py` and `pipeline.py` files for details on how this changes the pipeline topology across pipeline versions. You will create pipeline versions without and with tuning enabled in the subsequent lab exercises for comparison.

In [99]:
PIPELINE_NAME = "tfxcovertype"
# MODEL_NAME = "tfxcovertype_classifier"
# DATA_ROOT_URI = "gs://workshop-datasets/covertype/small"
# DATA_ROOT_URI = "/app/data"
DATA_ROOT_URI = "gs://qwiklabs-gcp-04-05130841e161/data"
TFX_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{PIPELINE_NAME}"
# RUNTIME_VERSION = "2.3"
# PYTHON_VERSION = "3.7"
# USE_KFP_SA = False
# ENABLE_TUNING = False
PIPELINE_JSON = f"{PIPELINE_NAME}.json"

In [103]:
%env PIPELINE_NAME={PIPELINE_NAME}
#%env MODEL_NAME={MODEL_NAME}
%env DATA_ROOT_URI={DATA_ROOT_URI}
%env TFX_IMAGE_URI={TFX_IMAGE_URI}
%env PIPELINE_JSON={PIPELINE_JSON}

env: PIPELINE_NAME=tfxcovertype
env: DATA_ROOT_URI=gs://qwiklabs-gcp-04-05130841e161/data
env: TFX_IMAGE_URI=gcr.io/qwiklabs-gcp-04-05130841e161/tfxcovertype
env: PIPELINE_JSON=tfxcovertype.json


In [104]:
!gcloud builds submit --timeout 15m --tag $TFX_IMAGE .

Creating temporary tarball archive of 14 file(s) totalling 126.2 KiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-gcp-04-05130841e161_cloudbuild/source/1638925122.074439-fda5e407693d4f93a619cbf12ddbbe0c.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-04-05130841e161/locations/global/builds/d43020e9-6456-4cc5-9ae6-7c10dd563ab9].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/d43020e9-6456-4cc5-9ae6-7c10dd563ab9?project=481343644334].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "d43020e9-6456-4cc5-9ae6-7c10dd563ab9"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-04-05130841e161_cloudbuild/source/1638925122.074439-fda5e407693d4f93a619cbf12ddbbe0c.tgz#1638925122506729
Copying gs://qwiklabs-gcp-04-05130841e161_cloudbuild/source/1638925122.074439-fda5e407693d4f93a619cbf12ddbbe0c.tgz#1638925122506729...
/ [1 files][ 18.5 KiB/ 18.5 KiB]                            

### Compile your pipeline code

You can build and upload the pipeline to the AI Platform Pipelines instance in one step, using the `tfx pipeline create` command. The `tfx pipeline create` goes through the following steps:
- (Optional) Builds the custom image to that provides a runtime environment for TFX components or uses the latest image of the installed TFX version 
- Compiles the pipeline code into a pipeline package 
- Uploads the pipeline package via the `ENDPOINT` to the hosted AI Platform instance.

As you debug the pipeline DSL, you may prefer to first use the `tfx pipeline compile` command, which only executes the compilation step. After the DSL compiles successfully you can use `tfx pipeline create` to go through all steps.

In [105]:
TFX_IMAGE_URI

'gcr.io/qwiklabs-gcp-04-05130841e161/tfxcovertype'

In [106]:
!tfx pipeline compile --engine vertex --pipeline_path runner.py

CLI
Compiling pipeline
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is not available: No module named 'tensorflow_decision_forests'
INFO:absl:struct2tensor is not available: No module named 'struct2tensor'
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.MutableMapping[str, typing.Any]
INFO:apache_beam.ty

Note: you should see a `{PIPELINE_NAME}.tar.gz` file appear in your current pipeline directory.

## Exercise: deploy your pipeline container to AI Platform Pipelines with TFX CLI

After the pipeline code compiles without any errors you can use the `tfx pipeline create` command to perform the full build and deploy the pipeline. You will deploy your compiled pipeline container hosted on Google Container Registry e.g. `gcr.io/[PROJECT_ID]/tfx_covertype_continuous_training` to run on AI Platform Pipelines with the TFX CLI.

In [107]:
aiplatform.init(project=PROJECT_ID, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name="tfxcovertype4",
    template_path=PIPELINE_JSON,
    enable_caching=False,
)

pipeline.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/481343644334/locations/us-central1/pipelineJobs/tfxcovertype-20211208011039
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/481343644334/locations/us-central1/pipelineJobs/tfxcovertype-20211208011039')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tfxcovertype-20211208011039?project=481343644334
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/481343644334/locations/us-central1/pipelineJobs/tfxcovertype-20211208011039 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/481343644334/locations/us-central1/pipelineJobs/tfxcovertype-2021

In [ ]:
# TODO: Your code here to use the TFX CLI to deploy your pipeline image to AI Platform Pipelines.

!tfx pipeline create  \
--pipeline_path=runner.py \
--endpoint={ENDPOINT} \
--build_target_image={TFX}

**Hint**: review the [TFX CLI documentation](https://www.tensorflow.org/tfx/guide/cli#create) on the "pipeline group" to create your pipeline. You will need to specify the `--pipeline_path` to point at the pipeline DSL and runner defined locally in `runner.py`, `--endpoint`, and `--build_target_image` arguments using the environment variables specified above.

Note: you should see a `build.yaml` file in your pipeline folder created by skaffold. The TFX CLI compile triggers a custom container to be built with skaffold using the instructions in the `Dockerfile`.

If you need to redeploy the pipeline you can first delete the previous version using `tfx pipeline delete` or you can update the pipeline in-place using `tfx pipeline update`.

To delete the pipeline:

`tfx pipeline delete --pipeline_name {PIPELINE_NAME} --endpoint {ENDPOINT}`

To update the pipeline:

`tfx pipeline update --pipeline_path runner.py --endpoint {ENDPOINT}`

### Create and monitor a pipeline run with the TFX CLI

After the pipeline has been deployed, you can trigger and monitor pipeline runs using TFX CLI.

*Hint*: review the [TFX CLI documentation](https://www.tensorflow.org/tfx/guide/cli#run_group) on the "run group".

In [ ]:
# TODO: your code here to trigger a pipeline run with the TFX CLI

!tfx run create --pipeline_name={PIPELINE_NAME} --endpoint={ENDPOINT}

To view the status of existing pipeline runs:

In [ ]:
!tfx run list --pipeline_name {PIPELINE_NAME} --endpoint {ENDPOINT}

To retrieve the status of a given run:

In [ ]:
RUN_ID = "[YOUR RUN ID]"

!tfx run status --pipeline_name {PIPELINE_NAME} --run_id {RUN_ID} --endpoint {ENDPOINT}

### Important 

A full pipeline run without tuning enabled will take about 40 minutes to complete. You can view the run's progress using the TFX CLI commands above or in the

## Exercise: deploy a pipeline version with tuning enabled

Incorporating automatic model hyperparameter tuning into a continuous training TFX pipeline workflow enables faster experimentation, development, and deployment of a top performing model.

The previous pipeline version read from hyperparameter default values in the search space defined in `_get_hyperparameters()` in `model.py` and used these values to build a TensorFlow WideDeep Classifier model.

Let's now deploy a new pipeline version with the `Tuner` component added to the pipeline that calls out to the AI Platform Vizier service for distributed and parallelized hyperparameter tuning. The `Tuner` component `"best_hyperparameters"` artifact will be passed directly to your `Trainer` component to deploy the top performing model. Review `pipeline.py` to see how this environment variable changes the pipeline topology. Also, review the tuning function in `model.py` for configuring `CloudTuner`.

Note that you might not want to tune the hyperparameters every time you retrain your model due to the computational cost. Once you have used `Tuner` determine a good set of hyperparameters, you can remove `Tuner` from your pipeline and use model hyperparameters defined in your model code or use a `ImporterNode` to import the `Tuner` `"best_hyperparameters"`artifact from a previous `Tuner` run to your model `Trainer`.


In [ ]:
ENABLE_TUNING = True

In [ ]:
%env ENABLE_TUNING={ENABLE_TUNING}

### Compile your pipeline code

In [ ]:
!tfx pipeline compile --engine kubeflow --pipeline_path runner.py

### Deploy your pipeline container to AI Platform Pipelines with the TFX CLI

In [ ]:
# TODO: your code to update your pipeline
!tfx pipeline update --pipeline_path runner.py --endpoint {ENDPOINT}

### Trigger a pipeline run from the Kubeflow Pipelines UI

On the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page, click `OPEN PIPELINES DASHBOARD`. A new browser tab will open. Select the `Pipelines` tab to the left where you see the `PIPELINE_NAME` pipeline you deployed previously. You should see 2 pipeline versions. 

Click on the most recent pipeline version with tuning enabled which will open up a window with a graphical display of your TFX pipeline directed graph. 

Next, click the `Create a run` button. Verify the `Pipeline name` and `Pipeline version` are pre-populated and optionally provide a `Run name` and `Experiment` to logically group the run metadata under before hitting `Start` to trigger the pipeline run.

## Important

A full pipeline run with tuning enabled will take about 50 minutes and can be executed in parallel while the previous pipeline run without tuning continues running. 

Take the time to review the pipeline metadata artifacts created in the GCS artifact repository for each component including data splits, your Tensorflow SavedModel, model evaluation results, etc. as the pipeline executes. In the GCP console, you can also view the Dataflow jobs for pipeline data processing as well as the AI Platform Training jobs for model training and tuning.

When your pipelines runs are complete, review your model versions on Cloud AI Platform Prediction and model evaluation metrics. Did your model performance improve with hyperparameter tuning?

## Next Steps

In this lab, you learned how to build and deploy a TFX pipeline with the TFX CLI and then update, build and deploy a new pipeline with automatic hyperparameter tuning. You practiced triggered continuous pipeline runs using the TFX CLI as well as the Kubeflow Pipelines UI.


In the next lab, you will construct a Cloud Build CI/CD workflow that further automates the building and deployment of the TensorFlow WideDeep Classifer pipeline code introduced in this lab.

## License

<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>